Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
print(src_port)

6432


In [6]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [7]:
SQL = f"""SELECT CONCAT(table_name,'.',column_name) as fields FROM information_schema.columns WHERE table_name IN ('contracts','internet','personal','phone') AND column_name != 'customer_id' AND column_name != 'index' ORDER BY fields"""
data_tables = pd.read_sql(SQL, src_conn)

flld_list = ','.join(data_tables['fields'].tolist())
print(flld_list)

contracts.begin_date,contracts.end_date,contracts.monthly_charges,contracts.paperless_billing,contracts.payment_method,contracts.total_charges,contracts.type,internet.device_protection,internet.internet_service,internet.online_backup,internet.online_security,internet.streaming_movies,internet.streaming_tv,internet.tech_support,personal.dependents,personal.gender,personal.partner,personal.senior_citizen,phone.multiple_lines


contracts.begin_date,contracts.end_date,contracts.monthly_charges,contracts.paperless_billing,contracts.payment_method,contracts.total_charges,contracts.type,internet.device_protection,internet.internet_service,internet.online_backup,internet.online_security,internet.streaming_movies,internet.streaming_tv,internet.tech_support,personal.dependents,personal.gender,personal.partner,personal.senior_citizen,phone.multiple_lines

In [8]:
# Пример выгрузки данных из БД
TABLE = 's1w1_electricity_access_percent'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, src_conn)

In [9]:
print(data.columns)
print(data.shape)

Index(['index', 'Country Name', 'Country Code', 'Indicator Name',
       'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965',
       '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974',
       '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017'],
      dtype='object')
(264, 63)


In [10]:
SQL = f"""
        SELECT CONCAT(table_name,'.',column_name) as fields 
            FROM information_schema.columns 
                WHERE 
                    table_name IN ('contracts','internet','personal','phone') AND 
                    column_name != 'customer_id' AND 
                    column_name != 'index' 
                ORDER BY fields
        """
fild_tables = pd.read_sql(SQL, src_conn)
fild_list   = ','.join(data_tables['fields'].tolist())
#print(fild_list)
sql_query = f"""
    SELECT contracts.customer_id,{fild_list}
    FROM contracts
        LEFT JOIN internet ON internet.customer_id = contracts.customer_id
        LEFT JOIN personal ON personal.customer_id = contracts.customer_id
        LEFT JOIN phone ON phone.customer_id = contracts.customer_id
    """
#print(sql_query)
data = pd.read_sql(sql_query, src_conn) #исполним написанный запрос
print(data.head())

  customer_id  begin_date             end_date  monthly_charges  \
0  7590-VHVEG  2020-01-01                   No            29.85   
1  5575-GNVDE  2017-04-01                   No            56.95   
2  3668-QPYBK  2019-10-01  2019-12-01 00:00:00            53.85   
3  7795-CFOCW  2016-05-01                   No            42.30   
4  9237-HQITU  2019-09-01  2019-11-01 00:00:00            70.70   

  paperless_billing             payment_method  total_charges            type  \
0               Yes           Electronic check          29.85  Month-to-month   
1                No               Mailed check        1889.50        One year   
2               Yes               Mailed check         108.15  Month-to-month   
3                No  Bank transfer (automatic)        1840.75        One year   
4               Yes           Electronic check         151.65  Month-to-month   

  device_protection internet_service online_backup online_security  \
0                No              DSL    

In [8]:
sql_query = f"""
    SELECT *
    FROM users_churn
    """
#print(sql_query)
data = pd.read_sql(sql_query, dst_conn) #исполним написанный запрос
print(data.head())

     id customer_id begin_date   end_date            type paperless_billing  \
0    18  8191-XWSZG 2015-10-01        NaT        One year                No   
1  5227  5924-IFQTT 2019-01-01        NaT  Month-to-month               Yes   
2  5229  7968-QUXNS 2019-08-01        NaT  Month-to-month                No   
3  5231  2919-HBCJO 2019-08-01 2019-12-01  Month-to-month               Yes   
4  5233  4236-XPXAV 2019-06-01 2019-11-01  Month-to-month               Yes   

              payment_method  monthly_charges  total_charges internet_service  \
0               Mailed check            20.65        1022.95             None   
1           Electronic check            72.80         930.05              DSL   
2               Mailed check            18.95         110.15             None   
3  Bank transfer (automatic)            76.65         333.60      Fiber optic   
4               Mailed check            99.15         465.05      Fiber optic   

   ... device_protection tech_support 

In [9]:
print(data.shape)

(7043, 22)
